In [9]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from deepface import DeepFace
from keras_facenet import FaceNet
from scipy.spatial.distance import cosine

In [10]:
my_images = glob.glob("../my_images2/*")

facenet = FaceNet()
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def image_embedding(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_detector.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        return None
    
    x, y, w, h = faces[0]
    face_crop = img[y:y+h, x:x+w]

    face_crop = cv2.resize(face_crop, dsize=(224, 224))
    face_crop = np.expand_dims(face_crop / 255.0, axis=0)

    embedding = facenet.embeddings(face_crop)
    return embedding[0], (x, y, w, h)

In [11]:
def extract_embeddings():
    embeddings = []
    for img_path in my_images:
        img = cv2.imread(img_path)
        try:
            embedding, _ = image_embedding(img)
            embedding = np.array(embedding)
            embeddings.append(embedding)
        except:
            print("No face detected.")
    return embeddings

embeddings = extract_embeddings()

1/1 [==============================] - 0s 40ms/step


In [12]:
def is_it_me(img):
    try:
        test_embedding, face_info = image_embedding(img)
    except:
        print("No face detected!")

    for emb in embeddings:
        similarity = 1 - cosine(emb, test_embedding)
        print(similarity)
        if similarity > 0.7:
            return test_embedding, True, face_info
        
    return test_embedding, False, face_info

test_img = cv2.imread("../test_images/test.jpg")
embedding, result, face_info = is_it_me(test_img)
print("This is you, Abolfazl!") if result else print("This is not you.")

1/1 [==============================] - 0s 71ms/step
1.0
This is you, Abolfazl!


In [13]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("Abolfazl Detector")

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    if cv2.waitKey(1) & 0xFF == 13:
        embedding, result, face_info = is_it_me(frame)

        if result:
            print(embedding[:5])
            x, y, w, h = face_info

            text = "This is you, Abolfazl!" if result else print("This is not you.")
            color = (0, 255, 0) if result else (0, 0, 255)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.8, color, 2, cv2.LINE_AA)
            
            cv2.imshow("Abolfazl Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 74ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 68ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 72ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 64ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 88ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 79ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 88ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
1/1 [==============================] - 0s 103ms/step
1.0
[ 0.01040818  0.01323743 -0.0813616   0.03703694 -0.00583532]
